In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load dataset
X = np.load("X.npy")
Y = np.load("Y.npy")

# Normalize images
X = preprocess_input(X)

# Split train/validation
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.1, random_state=42
)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode="nearest"
)

train_generator = datagen.flow(X_train, Y_train, batch_size=32)

# Load pretrained MobileNetV2 base
base_model = tf.keras.applications.MobileNetV2(
    input_shape=X.shape[1:],
    include_top=False,
    weights="imagenet"
)

# Freeze base layers
base_model.trainable = False

# Build transfer learning model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation="softmax")
])
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(Y),
    y=Y
)
class_weights = dict(enumerate(weights))
print(class_weights)


# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)   
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

# Train model
history = model.fit(
    train_generator,                     # Use augmented training data
    epochs=30,
    validation_data=(X_val, Y_val),      # Use proper validation set
    class_weight=class_weights,
    callbacks=[early_stop]
)
print(history.history)

{0: np.float64(1.0450785773366418), 1: np.float64(0.8406520292747838), 2: np.float64(1.0272357723577237), 3: np.float64(0.7090347923681257), 4: np.float64(0.873789764868603), 5: np.float64(3.0742092457420926)}
Epoch 1/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 76s 833ms/step - accuracy: 0.3333 - loss: 1.7434 - val_accuracy: 0.6364 - val_loss: 1.0765
Epoch 2/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 78s 771ms/step - accuracy: 0.5497 - loss: 1.1989 - val_accuracy: 0.6996 - val_loss: 0.8087
Epoch 3/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 55s 754ms/step - accuracy: 0.6258 - loss: 1.0188 - val_accuracy: 0.7549 - val_loss: 0.6913
Epoch 4/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 85s 793ms/step - accuracy: 0.6500 - loss: 0.9304 - val_accuracy: 0.7589 - val_loss: 0.6502
Epoch 5/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 57s 788ms/step - accuracy: 0.7036 - loss: 0.8209 - val_accuracy: 0.7945 - val_loss: 0.6056
Epoch 6/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 75s 688ms/step - accuracy: 0.7120 - loss: 0.7822 - val_accuracy: 0.7866 - val_loss: 0.5736
Epoch 7/30
72/72